# Load librabies

In [1]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch.optim as optim
import dgl
import random
import networkx as nx
import numpy as np
import scipy.sparse as sp
import itertools

/Users/aaronsteiner/Documents/GitHub/BugPlusEngine/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Data


In [2]:
# Load the graph and the deleted edge information from the file
with open("../../graphs_and_deleted_edges.pickle", "rb") as f:
    all_graphs = pickle.load(f)

In [3]:
print(all_graphs)

Graph(num_nodes=3, num_edges=12,
      ndata_schemes={'features': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'edge_features': Scheme(shape=(3,), dtype=torch.int64)})


In [4]:
# Split edge set for training and testing
g = all_graphs
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = 1
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]


In [5]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [18]:
# define the edge feature representation module
class EdgeModule(nn.Module):
    def __init__(self, in_feats, hidden_size):
        super(EdgeModule, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_feats, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
        )

    def forward(self, edges):
        return self.mlp(edges.data["edge_features"])

# initialize the edge feature module
edge_module = EdgeModule(in_feats=3, hidden_size=64)


In [24]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    
class EdgePredictionHead(nn.Module):
    def __init__(self, h_feats, out_feats):
        super(EdgePredictionHead, self).__init__()
        self.fc = nn.Linear(2 * h_feats, out_feats)

    def forward(self, g, h):
        # calculate source and destination features
        src_feat = h[g.srcnodes()]
        dst_feat = h[g.dstnodes()]
        # concatenate the source and destination features
        edge_feat = torch.cat([src_feat, dst_feat], dim=1)
        return self.fc(edge_feat)

# build the final model
model = nn.Sequential(GraphSAGE(in_feats=6, h_feats=64), EdgePredictionHead(h_feats=3, out_feats=32))


In [40]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_pos_g.edata["edge_features"] = torch.ones(train_pos_u.shape[0], 3)

train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
train_neg_g.edata["edge_features"] = torch.ones(train_neg_u.shape[0], 3)

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_pos_g.edata["edge_features"] = torch.ones(test_pos_u.shape[0], 3)

test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
test_neg_g.edata["edge_features"] = torch.ones(test_neg_u.shape[0], 3)


In [21]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [25]:
#model = GraphSAGE(6, 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [38]:
print(all_graphs.edata)
print(train_g.edata)

{'edge_features': tensor([[3, 3, 0],
        [2, 2, 0],
        [4, 4, 1],
        [4, 4, 1],
        [0, 0, 1],
        [0, 0, 1],
        [5, 5, 0],
        [3, 5, 0],
        [2, 5, 0],
        [4, 0, 1],
        [1, 1, 1],
        [1, 1, 1]])}
{'edge_features': tensor([[3, 3, 0],
        [2, 2, 0],
        [4, 4, 1],
        [0, 0, 1],
        [0, 0, 1],
        [5, 5, 0],
        [3, 5, 0],
        [2, 5, 0],
        [4, 0, 1],
        [1, 1, 1],
        [1, 1, 1]])}


In [47]:
print(train_pos_g.edata)

{'edge_features': tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])}


In [46]:
class EdgePredictor(nn.Module):
    def __init__(self, in_feats, edge_feats):
        super(EdgePredictor, self).__init__()
        self.fc = nn.Linear(in_feats + edge_feats, 1)

    def forward(self, g, h, edge_feat):
        h_g = dgl.mean_nodes(g, h)
        h_g = h_g.repeat(g.number_of_edges(), 1)
        return self.fc(torch.cat([h_g, edge_feat], dim=1))

class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Edge predictor
pred = EdgePredictor(64, 3)

# GraphSAGE model
model = GraphSAGE(6, 64)


# Optimizer
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# Training loop
for e in range(2_000):
    # Forward
    h = model(train_g, train_g.ndata["features"])
    pos_score = pred(train_pos_g, h, train_pos_g.edata["edge_features"])
    neg_score = pred(train_neg_g, h, train_neg_g.edata["edge_features"])
    loss = compute_loss(pos_score, neg_score)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

"""# Evaluation
with torch.no_grad():
    h = model(test_g, test_g.ndata["features"])
    pos_score = pred(test_pos_g, h, test_pos_g.edata["edge_features"])
    neg_score = pred(test_neg_g, h, test_neg_g.edata["edge_features"])
    print('AUC', compute_auc(pos_score, neg_score))"""


KeyError: tensor([[ -929.7772,  -439.4871,  -611.1336,  -504.5811,   985.9515, -1352.3868,
          -572.0422,   -80.4514,   708.3521,  1500.2485, -1062.7419,   584.3625,
            62.9555, -1210.4502,    39.4332,     6.2775,   456.4863,  1304.8599,
          1098.5679,  -413.2629,  -206.8904,  1294.6311,   104.2672,   380.7736,
         -1924.0911,   202.8310,   920.3375, -1583.1564,  -120.0067,  1333.7421,
          -716.2066, -1609.7782,  1116.8284,   189.1378, -1387.1909,  -592.5045,
          1654.8821,  -368.1522,  -843.9202,   184.1393, -1311.3763,   802.1870,
           714.5663,  -563.9583,  2794.1865, -2189.1594,  -344.3263, -1466.5177,
            17.8305,  -384.9168,   586.9098, -1317.5717,   605.0861,  -747.1750,
           462.0277,   642.3690,   139.8837, -1462.4358,  -893.5225, -1086.5804,
         -1099.9971,   767.9631,  -399.1743,   264.2786],
        [-1156.9180,    51.2558,   135.7433,  -303.1718,   914.8538, -1167.0736,
          -378.4260,  -109.4861,  -320.87

In [26]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(2_000):
    # forward
    h = model(train_g, train_g.ndata["features"])
    pos_score = pred(train_pos_g, h, e)
    neg_score = pred(train_neg_g, h, e)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

TypeError: Sequential.forward() takes 2 positional arguments but 3 were given

In [12]:
with torch.no_grad():
    test_g = train_g # graph to make predictions on
    h = model(test_g, test_g.ndata['features']) # compute hidden representation of nodes in test graph
    scores = pred(test_g, h) # make predictions using the scores function
    scores = scores.detach().numpy() # convert scores to numpy array

    # find the top 10 edges with highest scores
    top10 = np.argsort(scores)[::-1][:10]
    print('Top 10 edges with highest scores:')
    for i in top10:
        print('(%d, %d): %.4f' % (test_g.edges()[0][i], test_g.edges()[1][i], scores[i]))


Top 10 edges with highest scores:
(0, 1): -364.8163
(0, 1): -364.8163
(1, 0): -364.8163
(1, 0): -364.8163
(1, 0): -364.8163
(0, 2): -477.1709
(0, 2): -477.1709
(0, 2): -477.1709
(2, 1): -1671.4236
(2, 1): -1671.4236


In [13]:
all_graphs.ndata

{'features': tensor([[   0.,    2., -999., -999.,    3., -999.],
        [-999., -999., -999., -999., -999., -999.],
        [-999., -999., -999., -999., -999., -999.]])}

In [14]:
test_g

Graph(num_nodes=3, num_edges=11,
      ndata_schemes={'features': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'edge_features': Scheme(shape=(3,), dtype=torch.int64)})

In [20]:
# Make a prediction on the first node pair.
u, v = train_g.edges()
print('Edge features of the first edge:', train_g.edata[0])
print('Prediction score of the first edge:', pred(train_g, h)[0])


KeyError: 0

# Prepare training and testing sets

In [ ]:
# store the graph and the deleted edge information in a list
graphs = []
target = []


# Get all edges 
all_eids = all_graphs.edges()

# Convert the tensor to a list for easier iteration
all_eids = list(all_eids)

# Loop through all edge ids
for eid in range(len(all_eids[0])):
    # Get the source and destination node id
    u = all_eids[0][eid]
    v = all_eids[1][eid]
    # Append the edge to the deleted edges list
    target.append((u, v, all_graphs.edges[u, v].data))
    # Remove the edge from the graph
    all_graphs = dgl.remove_edges(all_graphs, eid)                     
    # Append the graph to the list
    graphs.append(all_graphs)
    # Add the edge back to the graph
    all_graphs = dgl.add_edges(all_graphs, u, v)

In [ ]:
target[:3]

[(tensor(1),
  tensor(0),
  {'src_port': tensor([2]), 'dst_port': tensor([2]), 'edge_type': tensor([0])}),
 (tensor(1),
  tensor(0),
  {'src_port': tensor([4]), 'dst_port': tensor([4]), 'edge_type': tensor([1])}),
 (tensor(0),
  tensor(2),
  {'src_port': tensor([0]), 'dst_port': tensor([0]), 'edge_type': tensor([0])})]

# The model

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
# Define the GCN message passing function
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # Compute the node representations by message passing
        g.ndata['h'] = inputs
        g.update_all(fn.copy_src('h', 'm'), fn.sum('m', 'h'))
        h = g.ndata.pop('h')
        return self.linear(h)

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h

In [ ]:
# Define the model, loss function, and optimizer
model = GCN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of training epochs
epochs = 100

for epoch in range(epochs):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(features, adjacency_matrix)
    
    # Compute the loss
    loss = criterion(outputs, labels)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

In [ ]:
# Split the data into training and validation sets
train_graphs = []
train_deleted_edges = []
val_graphs = []
val_deleted_edges = []

for graph, deleted_edge in zip(all_graphs, all_deleted_edges):
    if random.random() < 0.8:
        train_graphs.append(graph)
        train_deleted_edges.append(deleted_edge)
    else:
        val_graphs.append(graph)
        val_deleted_edges.append(deleted_edge)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your GCN model
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.ReLU(),
            nn.Linear(hidden_features, out_features)
        )
    
    def forward(self, x, adj):
        x = self.layers(x)
        x = torch.mm(adj, x)
        return x

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# in_features is the number of features of each node
in_features = graphs[0].number_of_nodes()
# hidden_features is the number of features of the hidden layer
hidden_features = 16
# out_features is the number of features of the output
out_features = 1

# Define the adjacency matrices
# Convert the DGL heterographs to NetworkX graphs
nx_graphs = [g.to_networkx() for g in graphs]

# Define the adjacency matrices
adjacency_matrices = [torch.tensor(nx.adjacency_matrix(g).todense()) for g in nx_graphs]
node_features = [torch.tensor(g.ndata) for g in graphs]


# Train the GCN model
model = GCN(in_features, hidden_features, out_features)
model.train()
num_epochs = 100
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = [model(g, adj) for g, adj in zip(graphs, adjacency_matrices)]
    outputs = torch.cat(outputs, dim=0)
    loss = criterion(outputs, target)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Print the loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluation
model.eval()
with torch.no_grad():
    outputs = [model(g, adj) for g, adj in zip(graphs, adjacency_matrices)]
    outputs = torch.cat(outputs, dim=0)
    loss = criterion(outputs, target)
    print(f'Final loss: {loss.item()}')


ValueError: could not determine the shape of object type 'HeteroNodeDataView'

In [ ]:
print(len(train_graphs), len(train_deleted_edges))

11 11


# Training